In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Conv2D,Flatten,Input,Dense,MaxPooling2D , BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import numpy as py
import random,shutil
from keras.preprocessing import image

In [2]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255),shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical'):
    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=(24,24))

In [3]:
BS=32
TS=(24,24)
train_batch=generator(r'C:\Users\Giridhar\Downloads\Drowsiness detection\prepared_data\train',shuffle=True,batch_size=BS,target_size=(24,24))

Found 80298 images belonging to 2 classes.


In [4]:
valid_batch=generator(r'C:\Users\Giridhar\Downloads\Drowsiness detection\prepared_data\test',shuffle=True,batch_size=BS,target_size=(24,24))
SPE=len(train_batch.classes)//BS
VS=len(valid_batch.classes)//BS
print(SPE,VS)

Found 4600 images belonging to 2 classes.
2509 143


In [5]:
model=Sequential([
    Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    Dropout(0.25),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')
    
])

In [6]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#vgg_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [8]:
checkpoint = ModelCheckpoint(r'C:\Users\Giridhar\Downloads\Drowsiness detection\models\model.h5',monitor='val_loss',save_best_only=True,verbose=3)


earlystop = EarlyStopping(monitor = 'val_loss',patience=7, verbose=3,restore_best_weights=True)
learning_rate =ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [9]:
model.fit_generator(train_batch,validation_data=valid_batch,epochs=1,steps_per_epoch=SPE,validation_steps=VS)

<ipython-input-9-84776d7d148f>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_batch,validation_data=valid_batch,epochs=1,steps_per_epoch=SPE,validation_steps=VS)


2509/2509 [==============================] - 400s 158ms/step - loss: 0.1332 - accuracy: 0.9510 - val_loss: 0.2570 - val_accuracy: 0.9047


In [10]:
model.save('models/cnnCat2.h5',overwrite=True)